In [97]:
import pandas as pd

# Only keep the target variables for the flights data and transaction data
flights = pd.read_csv("flights_grouped_master_without_los.csv", usecols=["FLIGHT_LEG_ARRIVAL_DATE", "TRIP_ORIGIN_COUNTRY", "flight_count"])
flights["FLIGHT_LEG_ARRIVAL_DATE"] = pd.to_datetime(
    flights["FLIGHT_LEG_ARRIVAL_DATE"], 
    dayfirst=True, 
    format='mixed', 
    errors='coerce' 
)

In [98]:
# Aggregate number of flights by date and country

flights_grouped = flights.groupby(["FLIGHT_LEG_ARRIVAL_DATE", "TRIP_ORIGIN_COUNTRY"])["flight_count"].sum().reset_index()
flights_grouped.rename(columns={
    "FLIGHT_LEG_ARRIVAL_DATE": "DATE",
    "TRIP_ORIGIN_COUNTRY": "COUNTRY",
    "flight_count": "TOTAL_FLIGHTS"
}, inplace=True)

In [101]:
print(flights_grouped.head())

        DATE COUNTRY  TOTAL_FLIGHTS
0 2022-01-01      AE           2789
1 2022-01-01      AF              2
2 2022-01-01      AG            339
3 2022-01-01      AL            666
4 2022-01-01      AM              1


In [103]:
# Only keep the target variables for the flights data and transaction data
transactions = pd.read_csv("RAW_LBS_TEAM1_20250428.csv", usecols=["CAL_DAY", "TRANX_TTL"])
transactions["CAL_DAY"] = pd.to_datetime(transactions["CAL_DAY"], dayfirst=True)


/var/folders/j9/2kb4w8sd2qn_ry70r38f81th0000gn/T/ipykernel_26668/955813260.py:3: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  transactions["CAL_DAY"] = pd.to_datetime(transactions["CAL_DAY"], dayfirst=True)


In [104]:
# Aggregate number of transactions by date and country
transactions_grouped = transactions.groupby(["CAL_DAY"])["TRANX_TTL"].sum().reset_index()
transactions_grouped.rename(columns={
    "CAL_DAY": "DATE",
    "TRANX_TTL": "TOTAL_TRANSACTIONS"
}, inplace=True)

In [105]:
print(transactions_grouped.head())

        DATE  TOTAL_TRANSACTIONS
0 2022-01-30                5914
1 2022-01-31                6318
2 2022-02-01                5632
3 2022-02-02                6073
4 2022-02-03                6818


In [106]:
# Merge the transaction and flight data by date
merged_by_date = pd.merge(transactions_grouped, flights_grouped, on="DATE", how="inner")

print(merged_by_date.head())

        DATE  TOTAL_TRANSACTIONS COUNTRY  TOTAL_FLIGHTS
0 2022-01-30                5914      AE           2710
1 2022-01-30                5914      AG            169
2 2022-01-30                5914      AL            912
3 2022-01-30                5914      AM              9
4 2022-01-30                5914      AO             17


In [107]:
# Check the country list
unique_countries = merged_by_date["COUNTRY"].unique()
print(unique_countries)

['AE' 'AG' 'AL' 'AM' 'AO' 'AR' 'AT' 'AU' 'AW' 'AZ' 'BB' 'BD' 'BE' 'BG'
 'BH' 'BJ' 'BL' 'BM' 'BR' 'BS' 'BW' 'BY' 'BZ' 'CA' 'CD' 'CH' 'CI' 'CL'
 'CM' 'CN' 'CO' 'CR' 'CU' 'CV' 'CW' 'CY' 'CZ' 'DE' 'DJ' 'DK' 'DO' 'DZ'
 'EC' 'EE' 'EG' 'ES' 'ET' 'FI' 'FJ' 'FR' 'GB' 'GD' 'GE' 'GH' 'GI' 'GM'
 'GN' 'GP' 'GR' 'HK' 'HN' 'HR' 'HU' 'ID' 'IE' 'IL' 'IN' 'IQ' 'IR' 'IS'
 'IT' 'JM' 'JO' 'JP' 'KE' 'KG' 'KH' 'KN' 'KR' 'KW' 'KY' 'KZ' 'LB' 'LC'
 'LK' 'LT' 'LU' 'LV' 'MA' 'MD' 'MG' 'MK' 'MM' 'MN' 'MQ' 'MT' 'MU' 'MV'
 'MW' 'MX' 'MY' 'NG' 'NL' 'NO' 'NP' 'NZ' 'OM' 'PA' 'PE' 'PH' 'PK' 'PL'
 'PR' 'PT' 'PY' 'QA' 'RE' 'RO' 'RS' 'RU' 'RW' 'SA' 'SC' 'SD' 'SE' 'SG'
 'SI' 'SK' 'SL' 'SN' 'SO' 'SS' 'ST' 'SV' 'SX' 'TC' 'TD' 'TH' 'TN' 'TR'
 'TT' 'TW' 'TZ' 'UA' 'UG' 'US' 'UY' 'UZ' 'VE' 'VG' 'VN' 'ZA' 'ZM' 'ZW'
 'BA' 'BN' 'DM' 'GU' 'ME' 'ML' 'MZ' 'NI' 'PF' 'TG' 'VC' 'VI' 'AF' 'BQ'
 'CF' 'GT' 'GY' 'LR' 'NE' 'BF' 'ER' 'GA' 'GW' 'LS' 'NC' 'BI' 'CG' 'MO'
 'PG' 'FO' 'TJ' 'AI' 'BO' 'GQ' 'MR' 'SR' 'KM' 'HT' 'GF' 'MS' 'TL' 'LA'
 'CK' 

In [108]:
def classify_country(code):
    if code in ['AE', 'SA', 'KW', 'QA', 'OM', 'BH']:
        return 'GCC'
    elif code in ['CN', 'HK', 'TW', 'MO']:
        return 'Greater China'
    elif code in ['AO', 'BJ', 'BW', 'CD', 'CI', 'CM', 'DJ', 'DZ', 'EG', 'ET', 'GA', 'GH', 'GM', 'GN',
                  'GQ', 'GW', 'KE', 'LR', 'LS', 'MG', 'ML', 'MR', 'MU', 'MW', 'MZ', 'NA', 'NE', 'NG',
                  'RE', 'RW', 'SC', 'SD', 'SH', 'SL', 'SN', 'SO', 'SS', 'ST', 'SZ', 'TD', 'TG', 'TN',
                  'TZ', 'UG', 'YT', 'ZA', 'ZM', 'ZW', 'CF', 'BI', 'CG', 'KM', 'ER']:
        return 'Africa'
    elif code in ['AF', 'AM', 'AZ', 'BD', 'BN', 'BT', 'GE', 'ID', 'IL', 'IN', 'IQ', 'IR', 'JO', 'JP',
                  'KG', 'KH', 'KP', 'KR', 'KZ', 'LA', 'LB', 'LK', 'MM', 'MN', 'MV', 'MY', 'NP', 'PH',
                  'PK', 'PS', 'SG', 'SY', 'TH', 'TJ', 'TL', 'TM', 'TR', 'UZ', 'VN', 'YE']:
        return 'Asia'
    elif code in ['AL', 'AT', 'BA', 'BE', 'BG', 'BY', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI',
                  'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IS', 'IT', 'LT', 'LU', 'LV', 'MD', 'ME', 'MK',
                  'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'RS', 'RU', 'SE', 'SI', 'SK', 'UA']:
        return 'Europe'
    elif code in ['AG', 'AI', 'BB', 'BM', 'BS', 'BZ', 'CA', 'CR', 'CU', 'DM', 'DO', 'GD', 'GT', 'HN',
                  'HT', 'JM', 'KN', 'LC', 'MX', 'NI', 'PA', 'PR', 'SV', 'SX', 'TC', 'TT', 'US', 'VC',
                  'VG', 'VI']:
        return 'North America'
    elif code in ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'GY', 'PE', 'PY', 'SR', 'UY', 'VE', 'GF']:
        return 'South America'
    elif code in ['AS', 'AU', 'CK', 'FJ', 'FM', 'GU', 'KI', 'MH', 'MP', 'NC', 'NF', 'NR', 'NU', 'NZ',
                  'PF', 'PG', 'PN', 'SB', 'TK', 'TL', 'TO', 'TV', 'VU', 'WS']:
        return 'Oceania'
    else:
        return 'Other'

In [109]:
merged_by_date["REGION_GROUP"] = merged_by_date["COUNTRY"].apply(classify_country)

In [110]:
print(merged_by_date.head())

        DATE  TOTAL_TRANSACTIONS COUNTRY  TOTAL_FLIGHTS   REGION_GROUP
0 2022-01-30                5914      AE           2710            GCC
1 2022-01-30                5914      AG            169  North America
2 2022-01-30                5914      AL            912         Europe
3 2022-01-30                5914      AM              9           Asia
4 2022-01-30                5914      AO             17         Africa


In [111]:
region_summary = merged_by_date.groupby(["REGION_GROUP", "DATE"])[["TOTAL_TRANSACTIONS", "TOTAL_FLIGHTS"]].sum().reset_index()
print(region_summary.head())

  REGION_GROUP       DATE  TOTAL_TRANSACTIONS  TOTAL_FLIGHTS
0       Africa 2022-01-30              201076           4815
1       Africa 2022-01-31              214812           3144
2       Africa 2022-02-01              197120           4808
3       Africa 2022-02-02              212555           4616
4       Africa 2022-02-03              252266           5126


In [112]:
# Check the relevance of each country on a daily basis (e.g. UK only)
country_daily = merged_by_date[merged_by_date["COUNTRY"] == "GB"]

correlation = country_daily["TOTAL_TRANSACTIONS"].corr(country_daily["TOTAL_FLIGHTS"])
print("The correlation coefficient between daily trading and flights in the UK is:", round(correlation, 4))

The correlation coefficient between daily trading and flights in the UK is: -0.0014


In [114]:
# Check the relevance of each region on a daily basis (e.g. Greater China only)
country_daily = region_summary[region_summary["REGION_GROUP"] == "Greater China"]

correlation = country_daily["TOTAL_TRANSACTIONS"].corr(region_summary["TOTAL_FLIGHTS"])
print("The correlation coefficient between daily trading and flights in Greater China the  is:", round(correlation, 4))

The correlation coefficient between daily trading and flights in Greater China the  is: 0.2411
